In [15]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell',set_css)

In [ ]:
!pip install transformers
!pip install newspaper3k

In [20]:
from transformers import pipeline
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.probability import FreqDist
from heapq import nlargest
nltk.download('punkt')
nltk.download('stopwords')
from newspaper import Article

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [23]:
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid th

In [24]:
def summarize(text, num_sentences=10):
    # tokenize the text into sentences and words
    sentences = sent_tokenize(text)
    words = word_tokenize(text.lower())

    # remove stop words and punctuation from the words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word.isalnum() and word not in stop_words]

    # calculate word frequency
    freq_dist = FreqDist(words)

    # rank sentences by their importance
    sentence_scores = {}
    for i, sentence in enumerate(sentences):
        sentence_words = word_tokenize(sentence.lower())
        sentence_score = 0
        for word in sentence_words:
            if word in freq_dist:
                sentence_score += freq_dist[word]
        sentence_scores[i] = sentence_score

    # select the top n sentences with highest scores to generate the summary
    summary_indexes = nlargest(num_sentences, sentence_scores, key=sentence_scores.get)
    summary_indexes.sort()
    summary = [sentences[i] for i in summary_indexes]
    return '.'.join(summary)

In [25]:
url = 'https://www.vox.com/the-goods/2023/2/7/23588382/gautam-adani-hindenburg-research-india-modi-short-seller'
article = Article(url)
article.download()
article.parse()
article.nlp()
title = article.title
summ = summarize(article.text)

In [26]:
print(summ)

On January 24, New York-based Hindenburg Research published a scathing 100-plus-page report making the case that Adani is “pulling the largest con in corporate history.” In it, Hindenburg, which was founded by activist short seller Nathan Anderson in 2017, alleges Adani Group has engaged in “brazen accounting fraud, stock manipulation, and money laundering” over the course of decades “with the help of enablers in government and a cottage industry of international companies that facilitate these activities.” It takes aim at seven companies listed by Adani that have seen their stock prices surge over the past three years..Shares of Adani’s flagship firm, Adani Enterprises, have plunged, as have the stock prices of other Adani companies..Much of Adani’s wealth is tied up in his companies, and as their prices have fallen, so has his net worth

Much of Adani’s wealth is tied up in his companies, and as their prices have fallen, so has his net worth..Adani has become one of the “poster child

In [27]:
summary_text = summarizer(summ, max_length=300, min_length=100, do_sample=False)[0]['summary_text']
print("\n\n\n")
print(summary_text)

Token indices sequence length is longer than the specified maximum sequence length for this model (776 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.9/dist-packages/transformers/generation/tf_utils.py:745: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(






a new report alleges that Adani is "pulling the largest con in corporate history" it takes aim at seven companies listed by Adani that have seen their stock prices surge . adani has become one of the "poster children" for india's economic growth prowess in recent years . an implosion of a conglomerate like Adani's would "dramatically raise" the country's risk profile, experts say .
